In [1]:
import pandas as pd #'version 1.2.4'
import numpy as np #'version 1.21.0'
from basico import * #'version 0.36'
from scipy.optimize import minimize #'version 1.10.0'
import random
import matplotlib.pyplot as plt #'version 3.3.4'
import os 

path = !pwd
path = path[0]


In [2]:
groups = 500
par_cores = 525

In [3]:
# define the number of cores to be used
# this code is commented because to prevent overloading and crashing computers. This code was run using AWS cloud computing.

# import ipyparallel as ipp # version 8.4.1
# cluster = ipp.Cluster(n=par_cores)
# cluster.start_cluster_sync()
# rc = cluster.connect_client_sync()
# rc.wait_for_engines(par_cores)
# dview = rc[0:par_cores]

Starting 525 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


 92%|#########1| 482/525 [00:00<?, ?engine/s]

In [4]:
# define the path for the models and the variant names then combine them
mod_path =path + "/Models/"
files =  ["NMGPL_Model_v3.xml", "NPMGL_Model_v3.xml", "NMPGL_Model_v3.xml", "NGPML_Model_v3.xml", "NGMPL_Model_v3.xml"] # ,"NMPGL_Model.xml", "NGMPL_Model.xml", "NGPML_Model.xml"]
models = [mod_path + m for m in files]

In [6]:
# define the parameters that will be fit independently. These are the binding rates
pars_to_fit = [ "Virion_Budding", "Virion_Assembly", "transcriptional_silencing",  "negative_strand_l_binding", "negative_strand_p_binding", "negative_strand_encapsidation", "positive_strand_encapsidation"]

In [5]:
# load one of the models in and separate reactions based on whether they are transcription reactions
# Genome synthesis reactions, or degradation reactions. 
# degredation reactions are split on whether they are genome, protein, or mRNA degradation reactions
mod = load_model(models[0])
set_current_model(mod)
reactions  = np.array(get_reaction_parameters()["reaction"])
transcription_reactions = reactions[["transcription_" in react for react in reactions]]
replication_reactions = reactions[["synthesis_" in react for react in reactions]]
deg_rs = reactions[["degradation" in react for react in reactions]]
genome_deg_rs = deg_rs[["sg" in react for react in deg_rs]]
deg_rs = deg_rs[((np.array(["sg" in react for react in deg_rs])-1) *-1) == 1]
mRNA_deg_rs = deg_rs[["m" in react for react in deg_rs]]
deg_rs = deg_rs[((np.array(["m" in react for react in deg_rs])-1) *-1) == 1]
prot_deg_rs = deg_rs

In [7]:
# load in models
models_loaded = []
for tmod in models:
    print(tmod)
    mod = basico.load_model(tmod)
    models_loaded.append(mod) # load base model in

/home/ubuntu/Paper_time/Fit_Parameter_Variants_SSA/Models/NMGPL_Model_v3.xml
/home/ubuntu/Paper_time/Fit_Parameter_Variants_SSA/Models/NPMGL_Model_v3.xml
/home/ubuntu/Paper_time/Fit_Parameter_Variants_SSA/Models/NMPGL_Model_v3.xml
/home/ubuntu/Paper_time/Fit_Parameter_Variants_SSA/Models/NGPML_Model_v3.xml
/home/ubuntu/Paper_time/Fit_Parameter_Variants_SSA/Models/NGMPL_Model_v3.xml


In [8]:
# record the current values of the reactions that will be adjusted as a unit
# this is important because we will be updating the model reaction rates iteratively
# this would be problematic with how we are adjusting the reaction rates.
rep_rates = []
tran_rates = []
genome_deg_rates = []
mRNA_deg_rates = []
prot_deg_rates = []
for i, model in enumerate(models_loaded):
    basico.set_current_model(model)
    ks = np.array(get_reaction_parameters()["value"])
    tr = ks[[react in transcription_reactions for react in reactions]]
    rr = ks[[react in replication_reactions for react in reactions]]
    gdr = ks[[react in genome_deg_rs for react in reactions]]
    mdr = ks[[react in mRNA_deg_rs for react in reactions]]
    pdr = ks[[react in prot_deg_rs for react in reactions]]
    
    
    
    tran_rates.append( tr )
    rep_rates.append( rr)
    genome_deg_rates.append( gdr )
    mRNA_deg_rates.append( mdr)
    prot_deg_rates.append( pdr )
    

In [9]:
data = np.array([4347, 2087, 2348, 1304, 3957]) # observed mean virions produced by variants seen in literature

In [36]:
# function that will be parallelized to simulate the model. 
# Runs 10 simulations and returns the number of viruses produced by them

def sim_data(model):
    import basico
    mod = basico.load_model(model)
    basico.set_current_model(mod)
    basico.set_model_unit(quantity_unit= "#")
    res = []
    for j in range(10):
        tc = basico.run_time_course(duration=57000, start_time=0, stepsize=1000, method="DirectMethod")
        res.append(tc["V"].loc[57000])
    basico.remove_datamodel(mod)
    return res

In [37]:
# make one list of all the reaction names
reactions = list(genome_deg_rs) + list(mRNA_deg_rs) + list(prot_deg_rs) + list(replication_reactions) + list(transcription_reactions) + pars_to_fit


In [38]:
# define temporary file names that will be made and then opened by the parallelized simulation function
model_temp = []
for i in range(5):
    model_temp.append(models[i].split(".")[0]+"_temp.xml")


In [39]:
# import the time package and set the varialbe track to zero. This will be used to track how many evaluations have passed.
import time
track = 0

# define the objective function
def obj_func(x):
    preds = np.zeros(len(models_loaded))
    # iterate through the models
    for i, model in enumerate(models_loaded):
        name = models[i].split("/")[-1]
        print(f"Simulating Model {name}")
        basico.set_current_model(model)
        # adjust the reaction rates that will be asjusted as a unit
        tr = tran_rates[i]/x[0]
        rr = rep_rates[i]/x[1]
        gdr = genome_deg_rates[i]/x[2]
        mdr = mRNA_deg_rates[i]/x[3]
        pdr = prot_deg_rates[i]/x[4]
        # make the full list of reaction names and full list of new parameters
        params = list(gdr) + list(mdr) + list(pdr) + list(rr) + list(tr) + list(x[5:])
        # now change the reaction rates
        for m, param_name in enumerate(reactions):
            basico.set_reaction(param_name, mapping={"k1": params[m]})
        # save the temporary model
        save_model(model_temp[i])
        # simulate the model. If an error occurs set the prediction value to infinity
        try:
            # this objective function parallelizes the simulations
            tc = []
            ar_map = dview.map_async(sim_data, [model_temp[i]] * groups)
            ar_map.wait_interactive()
            res = ar_map.get()
            tc += res
            preds[i] = np.mean(tc)
        except:
            preds[i] = np.inf
        finally:
            os.remove(model_temp[i])
    # calculate and return MSE
    test = np.mean((preds - data)**2)
    global track
    global start
    track += 1
    # these are for the sake of being able to track the optimization and see how long it is taking
    print("------------------------------------------------------------------")
    print()
    print(f"evaluation:{track}")
    print(f"Time Since Starting (s):{time.time() - start}")
    print(f"predictions: {preds}")
    print(f"test statistic: {test}")
    print()
    print("------------------------------------------------------------------")
    del m, preds, tc
    return test



In [15]:
# load in and parsearbitrary starting point
start_guess = pd.read_csv("vsv_fits_v1_2023_06_06.csv")
start_guess = start_guess[start_guess["obj function output"] == np.min(start_guess["obj function output"])]
start_guess = list(start_guess["parameter values"])[0].replace("[", "").replace("]", "").replace("\n", "").split(" ")
start_guess = [float(x) for x in start_guess]
start_guess = [1] * 5 + start_guess[5:]

In [19]:
# perform the optimization
tol = 10000
mi = 100
start = time.time()
res = minimize(obj_func, start_guess,method = 'Nelder-Mead', tol = tol, options = {"maxiter" : mi}) # perform optimization


Simulating Model NMGPL_Model_v3.xml


sim_data:   0%|          | 0/500 [00:00<?, ?tasks/s]

Simulating Model NPMGL_Model_v3.xml


sim_data:   0%|          | 0/500 [00:00<?, ?tasks/s]

Simulating Model NMPGL_Model_v3.xml


...

Simulating Model NGPML_Model_v3.xml


sim_data:   0%|          | 0/500 [00:00<?, ?tasks/s]

Simulating Model NGMPL_Model_v3.xml


sim_data:   0%|          | 0/500 [00:00<?, ?tasks/s]

------------------------------------------------------------------

evaluation:36
Time Since Starting (s):1798.9105007648468
predictions: [4225.423  1861.5054 2495.9776 1305.5854 3872.9924]
test statistic: 18917.188402168023

------------------------------------------------------------------


In [26]:
# update the parameters to include the other parameters fit as a unit.
pars_df = ["Transcription Rates", "Genome/Antigenome Synthesis Rates", "Genome/Antigenome Degradation Rates", "mRNA Degradation Rates", "Protein Degradation Rates"] + pars_to_fit

In [27]:
# create and save the new dataframe
functions_outputs = []
parameter_values = []
parameter_names = []
tolerance = []
maxiter = []
num_iter = []
functions_outputs.append(res.fun)
num_iter.append(res.nit)
parameter_values.append(res.x)
parameter_names.append(pars_df)
tolerance.append(tol)
maxiter.append(mi)
df = pd.DataFrame([functions_outputs, parameter_values, parameter_names, tolerance, maxiter, num_iter]).T
df.columns = ["obj function output", "parameter values", "parameter names", "tolerance", "maxiter", "number of iterations"]


In [28]:
df.to_csv("vsv_fits_SSA_v1_2023_06_21.csv")

In [41]:
# read it back in and collect the pit parameters
df = pd.read_csv("vsv_fits_SSA_v1_2023_06_21.csv", index_col = 0)
best_fit = df[df["obj function output"] ==  np.min(df["obj function output"])]
best_fit = list(best_fit["parameter values"])[0]
best_fit = best_fit.replace("[", "").replace("]", "").replace("\n", "").replace(" ", ",").split(",")
best_fit = [float(x) for x in best_fit]


In [ ]:

mod_path =path + "/Models/"
files =  ["NMGPL_Model_v3.xml", "NPMGL_Model_v3.xml", "NMPGL_Model_v3.xml", "NGPML_Model_v3.xml", "NGMPL_Model_v3.xml"] # ,"NMPGL_Model.xml", "NGMPL_Model.xml", "NGPML_Model.xml"]
models = [mod_path + m for m in files]

In [ ]:
# update parameters and save the models. Plot as well to see what is happening
data = np.array([4347, 2087, 2348, 1304, 3957])
preds = np.zeros(len(models_loaded))

for i, model in enumerate(models_loaded):
    basico.set_current_model(model)

    tr = tran_rates[i]/best_fit[0]
    rr = rep_rates[i]/best_fit[1]
    gdr = genome_deg_rates[i]/best_fit[2]
    mdr = mRNA_deg_rates[i]/best_fit[3]
    pdr = prot_deg_rates[i]/best_fit[4]            

    reactions = list(genome_deg_rs) + list(mRNA_deg_rs) + list(prot_deg_rs) + list(replication_reactions) + list(transcription_reactions) + pars_to_fit
    params = list(gdr) + list(mdr) + list(pdr) + list(rr) + list(tr) + list(best_fit[5:])

    for m, param_name in enumerate(reactions):
        basico.set_reaction(param_name, mapping={"k1": params[m]})
    save_model(model_temp[i])

        
    tc = []
    ar_map = dview.map_async(sim_data, [model_temp[i]] * groups)
    ar_map.wait_interactive()
    res = ar_map.get()
    tc += res
    print(np.mean(tc))
    preds[i] = np.mean(tc)
    new_mod = models[i].split(".")[0][:-2] + "SSA.xml"
    save_model(new_mod)
    os.remove(model_temp[i])
    
plt.scatter(range(len(preds)),preds, color = "blue")

plt.scatter(range(len(data)),data, color = "red")
plt.ylim(0,4500)

In [44]:
# make path for saving simulated dataset
result_path = path + "/stochastic_simulations/"
result_path

'/home/ubuntu/Paper_time/Fit_Parameter_Variants_SSA/stochastic_simulations/'

In [110]:
# function to be parallelized to get the full trajectories from the simulations
def get_full_data(y):
    import basico
    import pandas as pd
    model, x = y
    mod = basico.load_model(model)
    basico.set_current_model(mod)
    basico.set_model_unit(quantity_unit= "#")
    res = []
    for j in range(20):
        tc = basico.run_time_course(duration=57000, start_time=0, stepsize=100, method="DirectMethod")
        tc["model"] = model
        tc["iteration"] = (x * 20) + j + 1
        res.append(tc)
    basico.remove_datamodel(mod)
    return pd.concat(res)

In [83]:
# stochastic model names
stoch_models = [m.split(".")[0][:-2] + "SSA.xml" for m in models]

In [123]:
# cycle through the stochastic models and simulate 10000 simulations. Then save the trajectories.
for i, model in enumerate(stoch_models):
    par_list = list(zip([stoch_models[i]] * groups, range(groups)))
    par_list  = [list(x) for x in par_list]
    ar_map = dview.map_async(get_full_data,par_list)
    ar_map.wait_interactive()
    res = ar_map.get()
    res = pd.concat(res)
    res.to_csv(result_path + stoch_models[i].split("/")[-1].split(".")[0] + "_10000sims.csv")
    

get_full_data:   0%|          | 0/500 [00:00<?, ?tasks/s]

get_full_data:   0%|          | 0/500 [00:00<?, ?tasks/s]

get_full_data:   0%|          | 0/500 [00:00<?, ?tasks/s]

get_full_data:   0%|          | 0/500 [00:00<?, ?tasks/s]

get_full_data:   0%|          | 0/500 [00:00<?, ?tasks/s]